In [2]:
!pip install mysql-connector-python
!pip install PyMySQL

     -------------------------------------- 43.8/43.8 KB 305.3 kB/s eta 0:00:00


In [4]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pickle
import mysql.connector as MySQL 
import pymysql
import os

In [9]:
from sklearn.model_selection import cross_val_score,KFold
from sklearn.model_selection import cross_validate
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score,mean_absolute_error,mean_squared_error
from sklearn.ensemble import BaggingClassifier, GradientBoostingClassifier, AdaBoostClassifier, StackingClassifier
from sklearn import metrics
from sklearn import preprocessing
from collections import Counter
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.metrics import fbeta_score, f1_score,precision_score,recall_score,accuracy_score, roc_curve, auc
from sklearn.ensemble import RandomForestClassifier,RandomForestRegressor
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from xgboost import XGBClassifier
from sklearn.metrics import ConfusionMatrixDisplay, confusion_matrix
from sklearn.model_selection import GridSearchCV, train_test_split,RandomizedSearchCV
from numpy import float64, int64, number
from datetime import datetime
import json

C:\Users\HarshR-MBP_Windows\AppData\Local\Programs\Python\Python39\lib\site-packages\xgboost\compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index


In [5]:
# Supress Warnings
import warnings
warnings.filterwarnings('ignore')

In [6]:
pd.set_option("display.max_rows", None)
pd.set_option("display.max_columns", None)

In [ ]:
# host_name = "localhost"
# username = 'root'
# password = 'root' 
# db = "capstone"
# auth_plugin = 'mysql_native_password'
# connection = None

In [ ]:
# # DB connection
# def connect_to_db(host_name, username, password, auth_plugin):
#     try:
#         connection = MySQL.connect(host=host_name, user = username, password = password, auth_plugin=auth_plugin)
#         # preparing a cursor object
#         cursorObject = connection.cursor()
#         query = "SELECT Serial_number,document_id,URL,Credibility_rating, ad_count,ad_max_size,css_definitions,page_rank,bitly_clicks,bitly_referrers,tweets,delicious_bookmarks,fb_clicks,fb_comments,fb_likes,fb_shares,fb_total,alexa_linksin,alexa_rank,commas,dots,exclamations,questions,spelling_errors,text_complexity,smog,category,JJ,NN,DT,VB,RB,num_ne,sum_ne,document_url_y,X1,X2,X3,X4,X5,X9,Total,Leik,Eijk,Tastle,Leik_3_4_6,correction,resp_HNC,Controversial,troia_label FROM WEB_RAW_DATA"
#         df_raw = pd.read_sql(query,con=connection)
#         # disconnecting from server
#         connection.close()
#     except MySQL.errorcode as err:
#         if err.errno == errorcode.ER_TABLE_EXISTS_ERROR:
#             print("already exists.")
#         else:
#             print(err.msg)
#         else:
#             print('Connected!')
#     return df_raw

In [ ]:
# df_raw = connect_to_db(host_name, username, password, auth_plugin)

In [5]:
connection = pymysql.connect(
  host ="localhost",
  user ="root",
  passwd ="root",
  database = "capstone"
)
# preparing a cursor object
cursorObject = connection.cursor()
  
query = "SELECT Serial_number,document_id,URL,Credibility_rating, ad_count,ad_max_size,css_definitions,page_rank,bitly_clicks,bitly_referrers,tweets,delicious_bookmarks,fb_clicks,fb_comments,fb_likes,fb_shares,fb_total,alexa_linksin,alexa_rank,commas,dots,exclamations,questions,spelling_errors,text_complexity,smog,category,JJ,NN,DT,VB,RB,num_ne,sum_ne,document_url_y,X1,X2,X3,X4,X5,X9,Total,Leik,Eijk,Tastle,Leik_3_4_6,correction,resp_HNC,Controversial,troia_label FROM capstone.WEB_RAW_DATA"
cursorObject.execute(query)
columns = [col[0] for col in cursorObject.description]
records = cursorObject.fetchall()
   
# disconnecting from server
connection.close()

In [7]:
df_raw = pd.DataFrame(data=records,columns=columns)

In [11]:
df_raw.head(10)

,Serial_number,document_id,URL,Credibility_rating,ad_count,ad_max_size,css_definitions,page_rank,bitly_clicks,bitly_referrers,tweets,delicious_bookmarks,fb_clicks,fb_comments,fb_likes,fb_shares,fb_total,alexa_linksin,alexa_rank,commas,dots,exclamations,questions,spelling_errors,text_complexity,smog,category,JJ,NN,DT,VB,RB,num_ne,sum_ne,document_url_y,X1,X2,X3,X4,X5,X9,Total,Leik,Eijk,Tastle,Leik_3_4_6,correction,resp_HNC,Controversial,troia_label
0,1,2199424.0,http://unemployment.ohio.gov/,3.714285714285714,0E-15,0E-15,106.000000000000000,6.000000000000000,9.000000000000000,5.000000000000000,22.000000000000000,23.000000000000000,0E-15,10.000000000000000,12.000000000000000,62.000000000000000,84.000000000000000,68.000000000000000,6137.000000000000000,101.000000000000000,114.000000000000000,1.000000000000000,0E-15,98.000000000000000,6.543832913000000,15.903189010000000,computer_internet,121.000000000000000,938.000000000000000,184.000000000000000,338.000000000000000,52.000000000000000,7.000000000000000,29.000000000000000,gov,0E-15,1.000000000000000,2.000000000000000,2.000000000000000,2.000000000000000,0E-15,7.000000000000000,0.571428571000000,0.357142857000000,0.613196274000000,0.542857143000000,0E-15,2,U,3.000000000000000
1,2,2199427.0,http://www.law.cornell.edu/wex/unemployment_co...,4.428571428571429,4.000000000000000,75000.000000000000000,1824.000000000000000,5.000000000000000,12.000000000000000,1.000000000000000,1.000000000000000,0E-15,0E-15,0E-15,0E-15,0E-15,0E-15,9098.000000000000000,2661.000000000000000,15.000000000000000,23.000000000000000,0E-15,0E-15,2.000000000000000,3.813894731000000,15.903189010000000,business,35.000000000000000,168.000000000000000,59.000000000000000,71.000000000000000,5.000000000000000,4.000000000000000,17.000000000000000,edu,0E-15,0E-15,1.000000000000000,2.000000000000000,4.000000000000000,0E-15,7.000000000000000,0.714285714000000,0.714285714000000,0.735143276000000,0.600000000000000,0E-15,3,U,5.000000000000000
2,3,2547110.0,http://kiem-tv.com/?q=node/4395,4.000000000000000,0E-15,0E-15,18.000000000000000,2.000000000000000,0E-15,0E-15,5.000000000000000,0E-15,0E-15,0E-15,0E-15,2.000000000000000,2.000000000000000,185.000000000000000,809409.000000000000000,19.000000000000000,18.000000000000000,0E-15,1.000000000000000,26.000000000000000,3.163159951000000,11.208143260000000,health,18.000000000000000,100.000000000000000,23.000000000000000,52.000000000000000,11.000000000000000,5.000000000000000,20.000000000000000,com,0E-15,0E-15,2.000000000000000,3.000000000000000,2.000000000000000,0E-15,7.000000000000000,0.714285714000000,0.571428571000000,0.762835715000000,0.657142857000000,0E-15,2,U,4.000000000000000
3,4,2547112.0,http://www.sciencedaily.com/releases/2012/09/1...,4.833333333333333,9.000000000000000,75000.000000000000000,383.000000000000000,3.000000000000000,45.000000000000000,9.000000000000000,100.000000000000000,2.000000000000000,0E-15,17.000000000000000,35.000000000000000,113.000000000000000,165.000000000000000,50939.000000000000000,2549.000000000000000,19.000000000000000,20.000000000000000,0E-15,0E-15,15.000000000000000,3.727097973000000,14.554592550000000,health,39.000000000000000,156.000000000000000,46.000000000000000,67.000000000000000,21.000000000000000,8.000000000000000,24.000000000000000,com,0E-15,0E-15,0E-15,1.000000000000000,5.000000000000000,0E-15,6.000000000000000,0.916666667000000,0.916666667000000,0.892660382000000,0.866666667000000,0E-15,3,U,5.000000000000000
4,5,2547113.0,http://articles.timesofindia.indiatimes.com/20...,3.888888888888889,2.000000000000000,75000.000000000000000,599.000000000000000,3.000000000000000,3.000000000000000,3.000000000000000,7.000000000000000,0E-15,0E-15,1.000000000000000,3.000000000000000,3.000000000000000,7.000000000000000,1250.000000000000000,132.000000000000000,19.000000000000000,17.000000000000000,0E-15,0E-15,16.000000000000000,3.357533320000000,14.554592550000000,science_technology,32.000000000000000,112.000000000000000,39.000000000000000,57.00000000000000

In [36]:
from utils.dbutils import database
host_name = "localhost"
username = 'root'
password = 'root'  # dba password
db = "capstone"
table = 'web_raw_data'
auth_plugin = 'mysql_native_password'
table = "web_raw_data"


ModuleNotFoundError: No module named 'utils'

In [10]:
def regression_retrain():
  #removing the obvious column not going to used in the model
    cols_not_required = ['Serial_number','document_id','URL','category','ad_count', 'ad_max_size', 'css_definitions', 'page_rank',
       'bitly_clicks', 'bitly_referrers', 'tweets', 'delicious_bookmarks',
       'fb_clicks', 'fb_comments', 'fb_likes', 'fb_shares', 'fb_total',
       'alexa_linksin','num_ne', 'sum_ne','X1', 'X2', 'X3', 'X4', 'X5', 'X9', 'Total',
       'Leik', 'Eijk', 'Tastle','Leik_3_4_6', 'correction', 'resp_HNC', 'Controversial', 'troia_label']
    # cols_not_required = ['document_id','URL']
    # make a copy of the raw data
    df_main = df_raw.copy()
    df_main.drop(cols_not_required,axis=1,inplace=True)
    
    #getting on the popular domain as in the list below
    array = ['com', 'org', 'gov', 'edu','net','uk','au','ca','in']
    df2 = df_main.loc[df_main['document_url_y'].isin(array)]
    print(df2.columns)
    # Dropping the missing values.
    df2=df2.fillna(0)
    
    # Using label encoder to convert document_url_y into dummy
    le = preprocessing.LabelEncoder()
    df2.document_url_y = le.fit_transform(df2.document_url_y)
    df1=df2

    # defining x and y
    y = df1["Credibility_rating"]
    X = df1.drop("Credibility_rating",axis=1) 

    # Splitting the dataset
    X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.3,random_state=0)
    
    # performing preprocessing part
    sc = StandardScaler()
    X_train = sc.fit_transform(X_train)
    X_test = sc.transform(X_test)

    #reading previous metrics 

    path_of_file = '../models/regression_performance.json'
    # checking if size of file is 0
    if os.path.getsize(path_of_file) == 0:
        return "File is empty, train the inital model"
    else:
        #loading the json 
        with open(path_of_file, 'r') as f:
            previous_perf = json.load(f)
            if(len(previous_perf)!=0):
                previous_r2_score = previous_perf['test_r2_score']
            else :
                return "train the inital model Or Check the Json"
        f.close()

    # Random Forest model
    rf = RandomForestRegressor()
    rf_model = rf.fit(X_train,y_train)
    #y_train_pred = rf_model.predict(X_train)
    y_test_pred = rf_model.predict(X_test)

    # train r2_score = r2_score(y_train,y_train_pred)
    test_r2_score = r2_score(y_test,y_test_pred)
    ac_score=1-mean_absolute_error(y_test,y_test_pred)
    perform_metrics ={'test_r2_score':test_r2_score}
    
    # compare old and new accuracy and then pickle the new model to use by the application, 
    # if retrained model gives better performance
    if test_r2_score>previous_r2_score:
        with open("../models/le.pkl",'wb') as file:
            pickle.dump(le,file)
        file.close()
        with open("../models/standardscaler.pkl",'wb') as file:
            pickle.dump(sc,file)
        file.close()
        with open("../models/category_model.pkl",'wb') as file:
            pickle.dump(rf_model,file)
        file.close()
        # storing the data in JSON format
        current_perf={"test_r2_score":test_r2_score,"accuracy_score":ac_score}
        # saving the upgraded accuracy along with other metrics and old values in Json
        with open(path_of_file, "w") as outfile:
            json.dump(current_perf, outfile)
        outfile.close()
        return current_perf

    return previous_perf


In [11]:
regression_retrain()

Index(['Credibility_rating', 'alexa_rank', 'commas', 'dots', 'exclamations',
       'questions', 'spelling_errors', 'text_complexity', 'smog', 'JJ', 'NN',
       'DT', 'VB', 'RB', 'document_url_y'],
      dtype='object')


{'test_r2_score': 0.16085717880750294, 'accuracy_score': 0.40559166211766673}

In [12]:
def classification_retrain():
    
    #removing the obvious column not going to used in the model
    cols_not_required = ['Serial_number','document_id','URL','Credibility_rating']
    
    # make a copy of the raw data
    df_main = df_raw.copy()
    df_main.drop(cols_not_required,axis=1,inplace=True)
    
    #getting on the popular domain as in the list below
    array = ['com', 'org', 'gov', 'edu','net','uk','au','ca','in']
    df2 = df_main.loc[df_main['document_url_y'].isin(array)]
    
    #filling all null category as 'unknown'
    df2["category"].fillna("Unknown", inplace = True)
    
    # Dropping the missing values.
    df2=df2.fillna(0)
    
    # Categorical features has to be converted into integer values for the model to process(one hot encoding).
    y_data = df2["category"]
    X_data = df2.drop(['category','ad_count', 'ad_max_size', 'css_definitions', 'page_rank',
       'bitly_clicks', 'bitly_referrers', 'tweets', 'delicious_bookmarks',
       'fb_clicks', 'fb_comments', 'fb_likes', 'fb_shares', 'fb_total',
       'alexa_linksin','num_ne', 'sum_ne','X1', 'X2', 'X3', 'X4', 'X5', 'X9', 'Total',
       'Leik', 'Eijk', 'Tastle','Leik_3_4_6', 'correction', 'resp_HNC', 'Controversial', 'troia_label'],axis=1)
    print(X_data.columns)
    # select categorical features
    cat_x = X_data.select_dtypes(include=['object']).columns
    
    le_x_docURL = preprocessing.LabelEncoder()
    X_data.document_url_y = le_x_docURL.fit_transform(X_data.document_url_y)
    
    # label encode the target variable
    le_y = preprocessing.LabelEncoder()
    y = le_y.fit_transform(y_data)

    # Splitting the dataset
    X_train,X_test,y_train,y_test = train_test_split(X_data,y,test_size=0.3,random_state=0)
    
    # performing preprocessing part
    sc = StandardScaler()
    X_train = sc.fit_transform(X_train)
    X_test = sc.transform(X_test)
    
    #reading previous metrics 

    path_of_file = '../models/classification_performance.json'
    # checking if size of file is 0
    if os.path.getsize(path_of_file) == 0:
        return "File is empty, train the inital model"
    else:
        #loading the json 
        with open(path_of_file, 'r') as f:
            previous_perf = json.load(f)
            if(len(previous_perf)!=0):
                previous_f1=previous_perf['F1-Score']
            else :
                return "train the inital model Or Check the Json"
        f.close()
      
    # Random Forest model
    rf =RandomForestClassifier()
    rf_model = rf.fit(X_train,y_train)
    #y_train_pred = rf_model.predict(X_train)
    y_test_pred = rf_model.predict(X_test)

#     precision = precision_score(y_test,y_test_pred, average='weighted')
#     recall = recall_score(y_test,y_test_pred, average='weighted')
    accuracy = accuracy_score(y_test,y_test_pred)
    f1 = f1_score(y_test,y_test_pred, average='weighted')
    perform_metrics ={"Accuracy" : accuracy,"F1-score" : f1}
    
    # compare old and new accuracy and then pickle the new model to use by the application, 
    # if retrained model gives better performance
    if f1>previous_f1:
        with open("../models/le_x_docURL.pkl",'wb') as file:
            pickle.dump(le_x_docURL,file)
        file.close()
        with open("../models/le_y.pkl",'wb') as file:
            pickle.dump(le_y,file)
        file.close()
        with open("../models/sscaler_class.pkl",'wb') as file:
            pickle.dump(sc,file)
        file.close()
        with open("../models/category_model.pkl",'wb') as file:
            pickle.dump(rf_model,file)
        file.close()
        # storing the data in JSON format
        current_perf={"Accuracy":accuracy,"F1-Score":f1}
        #saving the upgraded accuracy along with other metrics and old values in Json
        with open(path_of_file, "w") as outfile:
            json.dump(current_perf, outfile)
        outfile.close()
        return current_perf
    
    return previous_perf

In [13]:
classification_retrain()
#before running this, run the primary training of the model for the same json (classification_performance) for initial accuracy

C:\Users\HarshR-MBP_Windows\AppData\Local\Temp\ipykernel_6928\1677671040.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2["category"].fillna("Unknown", inplace = True)


Index(['alexa_rank', 'commas', 'dots', 'exclamations', 'questions',
       'spelling_errors', 'text_complexity', 'smog', 'JJ', 'NN', 'DT', 'VB',
       'RB', 'document_url_y'],
      dtype='object')


{'Accuracy': 0.45786013150029886, 'F1-Score': 0.428578395969731}

In [44]:
classification_retrain()

{'Accuracy': 0.46172248803827753, 'F1-Score': 0.4404065503339959}